In [7]:
import numpy as np

rho_0 = .8309 # g/cc
rho_0_err = .0007

U_s = 7.039 # km/s
U_s_err = .001

u_p = 3.2165 # km/s, actual
u_p_err = .004 #estimated

u_apparent = 3.234
u_app_err = .008 #estimated

n_0 = 1.462 #ambient index of refraction
n_0_err = .0005 #assumed



C_L = 6.715646 #lagrangian sound speed
C_L_err = .05





In [8]:
#pressure
P = rho_0*U_s*u_p

P_err = np.sqrt((U_s*u_p*rho_0_err)**2 + (rho_0*u_p*U_s_err)**2 + (rho_0*U_s*u_p_err)**2)

print(P)
print(P_err)

18.812359954149997
0.02838380517326691


In [9]:
#shocked density
rho_1 = (rho_0*U_s)/(U_s - u_p)

pd_1 = U_s/(U_s - u_p) #partial deriv w/rt rho_0
pd_2 = ((U_s - u_p)*rho_0 + (rho_0*U_s*u_p))/((U_s - u_p)**2) # " w/rt U_s
pd_3 = (rho_0*U_s)/((U_s - u_p)**2) # " w/rt u_p

rho_1_err = np.sqrt((pd_1*rho_0_err)**2 + (pd_2*U_s_err)**2 + (pd_3*u_p_err)**2)

print(rho_1)
print(rho_1_err)

1.530073276651406
0.0025475140054271534


In [10]:
#density compression ratio
rho_ratio = U_s/(U_s - u_p) # rho_1/rho_0

rho_ratio_err = ((u_p*U_s)/((U_s - u_p)**2))*np.sqrt((U_s_err/U_s)**2 + (u_p_err/u_p)**2)

print(rho_ratio)
print(rho_ratio_err)

1.8414650098103336
0.0019395076789241485


In [11]:
#index of refraction and window correction

n = (u_apparent - n_0*U_s)/(u_p - U_s)
a = u_apparent/u_p #window correction

a_err = np.sqrt((u_app_err/u_apparent)**2 + (u_p_err/u_p)**2)

n1 = 1/(u_p - U_s)
n2 = U_s/(u_p - U_s)
n3 = (U_s*n_0 - u_apparent)/(u_p - U_s)**2
n4 = (u_apparent - n_0*u_p)/(u_p - U_s)**2

n_err = np.sqrt((u_app_err*n1)**2 + (n_0_err*n2)**2 + (u_p_err*n3)**2 + (U_s_err*n4)**2)

print(n)
print(n_err)
print("\n")
print(a)
print(a_err)

1.8461786788750816
0.002995033232098009


1.0054406964091405
0.0027687154240700258


In [31]:
#Eulerian sound speed
C_E = C_L/rho_ratio
C_E_err = np.sqrt((C_L_err/C_L)**2 + (rho_ratio_err/rho_ratio)**2)

print(C_E)
print(C_E_err)

5.524507131237054
0.007459518831058562


In [8]:
import matplotlib.pyplot as plt
from matplotlib import colors
import itertools

Ps = []
rhos = []


def monte():
    for _ in itertools.repeat(None, 100000):
        P_ = np.random.normal(rho_0, rho_0_err)*np.random.normal(U_s, U_s_err)*np.random.normal(u_p, u_p_err)
        Ps.append(P_)
        
        rho_1_ = (np.random.normal(rho_0, rho_0_err)*np.random.normal(U_s, U_s_err))/(np.random.normal(U_s, U_s_err) - np.random.normal(u_p, u_p_err))
        rhos.append(rho_1_)
       
        
monte()
print("Mean P: " + str(format(np.mean(Ps), '.4f')) + "\nP Standard deviation: " + str(format(np.std(Ps), '.4f')))
print("Mean rho: " + str(format(np.mean(rhos), '.4f')) + "\nRho Standard deviation: " + str(format(np.std(rhos), '.4f')))

"""
plt.hist2d(rhos, Ps, bins=100, norm=colors.LogNorm())
plt.xlabel(r"$\rho_1$")
plt.ylabel("P")
plt.show()
"""

Mean P: 10.1438
P Standard deviation: 0.0194
Mean rho: 1.3660
Rho Standard deviation: 0.0035


'\nplt.hist2d(rhos, Ps, bins=100, norm=colors.LogNorm())\nplt.xlabel(r"$\rho_1$")\nplt.ylabel("P")\nplt.show()\n'

In [14]:
#Monte Carlo error for impedance matching
import matplotlib.pyplot as plt
from matplotlib import colors
import itertools
from scipy.optimize import fsolve
import math

#enter TPX values in top cell, then adjust these if needed (only density and v_proj should need adjusting)

Ps = []
ups = []
rhos = []

rho_0_Cu = 8.935
rho_0_Cu_err = .002
v_proj = 3.670
v_proj_err = .0025

#do not change these values, they're from Marcus's alpha quartz paper
C = 3.970
C_err = np.sqrt(2.9250*10**-4)
S = 1.479
S_err = np.sqrt(5.0020*10**-4)
CS_err = 3.7920*10**-4 #correlation
r = CS_err/(C_err*S_err) #correlation coefficient

#run once before starting monte carlo so that the values aren't undefined.
z1 = np.random.normal(0, 1, 1) #mean 0, stdev 1, 1 value
z2 = np.random.normal(0, 1, 1)
C_ = np.random.normal(C, C_err)
S_ = np.random.normal(S, S_err)
C_1 = C_ + C_err*(r*z2 + (z1*np.sqrt(1-r**2)))
S_1 = S_ + S_err*z2
rhozero = np.random.normal(rho_0, rho_0_err)
shockspeed = np.random.normal(U_s, U_s_err)
rhozeroCu = np.random.normal(rho_0_Cu, rho_0_Cu_err)



def equations(q):
    u_, P_ = q
    #enter equations, solved for zero
    return (rhozero*shockspeed*u_ - P_, rhozeroCu*(C_1*(np.random.normal(v_proj, v_proj_err) - u_) + S_1*(np.random.normal(v_proj, v_proj_err) - u_)**2) - P_)



def monte():
    for _ in itertools.repeat(None, 1000000):
        z1 = np.random.normal(0, 1, 1) #mean 0, stdev 1, 1 value
        z2 = np.random.normal(0, 1, 1)

        C_ = np.random.normal(C, C_err)
        S_ = np.random.normal(S, S_err)

        C_1 = C_ + C_err*(r*z2 + (z1*np.sqrt(1-r**2)))
        S_1 = S_ + S_err*z2
        #now C_1 and S_1 take correlation and variability into account. Use these in the rest of the calculation.

        rhozero = np.random.normal(rho_0, rho_0_err)
        shockspeed = np.random.normal(U_s, U_s_err)
        rhozeroCu = np.random.normal(rho_0_Cu, rho_0_Cu_err)
        
        u_, P_ =  fsolve(equations, (3.2, 18.7)) #guesses
        Ps.append(P_)
        ups.append(u_)
        
        rho_1_ = (np.random.normal(rho_0, rho_0_err)*np.random.normal(U_s, U_s_err))/(np.random.normal(U_s, U_s_err) - u_)
        rhos.append(rho_1_)

monte()
print("Mean P: " + str(format(np.mean(Ps), '.4f')) + "\nP Standard deviation: " + str(format(np.std(Ps), '.4f')))
print("Mean up: " + str(format(np.mean(ups), '.4f')) + "\nup Standard deviation: " + str(format(np.std(ups), '.4f')))
print("Mean rho_1: " + str(format(np.mean(rhos), '.4f')) + "\nRho Standard deviation: " + str(format(np.std(rhos), '.4f')))



C:\Users\lauren.barmore\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Mean P: 18.7042
P Standard deviation: 0.0141
Mean up: 3.2001
up Standard deviation: 0.0024
Mean rho_1: 1.5235
Rho Standard deviation: 0.0017


In [8]:
#impedance matching exact calculation (not including covariance matrix elements)
#THIS IS NOT CORRECT DO NOT USE. USE calculator in TPX Paper Calculations file instead
def impedance_match(rho_0_TPX, Us, rho_0_Cu, v_proj, C_TPX, S_TPX, C_Cu, S_Cu):
    #solve quadratic equation for up
    a = rho_0_TPX*S_TPX - rho_0_Cu*S_Cu
    b = rho_0_TPX*C_TPX + rho_0_Cu*C_Cu + 2*rho_0_Cu*S_Cu*v_proj
    c = -rho_0_Cu*C_Cu*v_proj - rho_0_Cu*S_Cu*v_proj**2
    
    up1 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
    up2 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)
    
    return up1, up2

def find_P(rho_0_TPX, Us, up):
    #solve RH jump condition for P
    P = rho_0_TPX*Us*up
    return P

#TPX C, S values from linear fit to my data, others from shot 18-2s19
up_1, up_2 = impedance_match(.8306,7.03902,8.935,3.670, 1.942968,1.622787,3.970,1.479)
print(up_1, up_2)

P_1 = find_P(.8306, 7.03902, up_1)
print(P_1)


3.2107637082870437 8.088059546445425
18.77208324303728
